In [238]:
import pandas as pd
import json

# Load the JSON file
with open("../etl/extract/data/raw/parks.json", "r") as f:
    parks_data = json.load(f)
# Normalize the parks list
parks_df = pd.json_normalize(
    parks_data,
    record_path='parks',          # list of parks
    meta=['id', 'name'],          # top-level company metadata
    meta_prefix='company_'        # add prefix to top-level metadata
)
# Rename columns for clarity
parks_df = parks_df.rename(columns={
    'id': 'park_id',
    'name': 'park_name'
})
# Drop unnecessary columns
parks_df = parks_df.drop(columns=['company_id', 'company_name', 'timezone'])

# Set index if 'park_id' exists
#if 'id' in parks_df.columns:
parks_df.set_index('park_id', inplace=True)

parks_df

,park_name,country,continent,latitude,longitude
park_id,,,,,
57,California's Great America,United States,North America,37.397799,-121.974717
58,Canada's Wonderland,Canada,North America,43.843,-79.539
59,Carowinds,United States,North America,35.1045,-80.9394
50,Cedar Point,United States,North America,41.4822,-82.6835
69,Dorney Park,United States,North America,40.577942,-75.531528
...,...,...,...,...,...
5,Epcot,United States,North America,28.374694,-81.549404
30,Shanghai Disney Resort,China,Asia,31.144,121.657
274,Tokyo Disneyland,Japan,Asia,35.634848,139.879295


In [ ]:
park_location_df = parks_df.filter(['park_id', 'park_name', 'continent'])
park_location_df.drop_duplicates(subset=['park_name'], inplace=True)
park_location_df = park_location_df.groupby(['continent']).agg(num_of_parks=('park_name', 'count'))
park_location_df.to_csv('parks_by_continent.csv', index=True)



In [275]:
park_location_df = full_df.filter(['park_id', 'continent']).reset_index()
park_location_df.drop_duplicates(subset=['park_id'], inplace=True)
park_location_df = park_location_df.groupby(['continent']).agg(num_of_parks=('park_id', 'count'))
park_location_df



,num_of_parks
continent,
Asia,15
Europe,41
North America,64
South America,1


In [229]:

# Load the JSONL queue times file

file_path = "../etl/extract/data/raw/queue_times_log.jsonl"
df_raw = pd.read_json(file_path, lines=True)
records = []

for _, row in df_raw.iterrows():
    timestamp = row["timestamp"]
    park_id = row["park_id"]
    data = row["data"]

    if data["lands"] == []:
        for ride in data.get("rides", []):
            record = {
                "timestamp": timestamp,
                "park_id": park_id,
                "ride_id": ride.get("id"),
                "ride_name": ride.get("name"),
                "is_open": ride.get("is_open"),
                "wait_time": ride.get("wait_time"),
                "last_updated": ride.get("last_updated"),
                "land_id": None,
                "land_name": None
            }
        records.append(record)
    else:
        for land in data.get("lands", []):
         
            land_id = land.get("id")
            land_name = land.get("name")
            for ride in land.get("rides", []):
                record = {
                    "timestamp": timestamp,
                    "park_id": park_id,
                    "ride_id": ride.get("id"),
                    "ride_name": ride.get("name"),
                    "is_open": ride.get("is_open"),
                    "wait_time": ride.get("wait_time"),
                    "last_updated": ride.get("last_updated"),
                    "land_id": land_id,
                    "land_name": land_name
                }
            records.append(record)

df_rides = pd.DataFrame(records)
df_rides

,timestamp,park_id,ride_id,ride_name,is_open,wait_time,last_updated,land_id,land_name
0,2025-05-13 18:41:52.601463+00:00,57,5852,RailBlazer,False,0,2025-05-13T18:40:22.000Z,9.0,Coasters
1,2025-05-13 18:41:52.601463+00:00,57,7720,Woodstock Express,False,0,2025-05-13T18:40:22.000Z,11.0,Family
2,2025-05-13 18:41:52.601463+00:00,57,7660,The Pumpkin Patch,False,0,2025-05-13T18:40:22.000Z,13.0,Kids
3,2025-05-13 18:41:52.601463+00:00,57,7668,Xtreme Skyflyer,False,0,2025-05-13T18:40:22.000Z,10.0,Thrill
4,2025-05-13 18:41:52.601463+00:00,57,13057,Tide Pool Slides,False,0,2025-05-13T18:40:22.000Z,14.0,Water Park
...,...,...,...,...,...,...,...,...,...
9515,2025-05-14 12:12:39.894646+00:00,274,7991,Westernland Shootin' Gallery,False,0,2025-05-14T12:07:54.000Z,NaN,None
9516,2025-05-14 12:12:40.026123+00:00,275,8034,Venetian Gondolas,False,0,2025-05-14T12:08:09.000Z,NaN,None
9517,2025-05-14 12:12:40.172345+00:00,28,10846,Spider-Man W.E.B. Adventure Single Rider,True,5,2025-05-14T12:10:04.000Z,906.0,Marvel Avengers Campus
9518,2025-05-14 12:12:40.172345+00:00,28,40,The Twilight Zone Tower of Terror,True,30,2025-05-14T12:10:04.000Z,26.0,Production Courtyard


In [239]:
full_df = df_rides.merge(parks_df, left_on='park_id', right_index=True, how='left')
full_df = full_df.drop(columns=['land_id', 'land_name'])
full_df

,timestamp,park_id,ride_id,ride_name,is_open,wait_time,last_updated,park_name,country,continent,latitude,longitude
0,2025-05-13 18:41:52.601463+00:00,57,5852,RailBlazer,False,0,2025-05-13T18:40:22.000Z,California's Great America,United States,North America,37.397799,-121.974717
1,2025-05-13 18:41:52.601463+00:00,57,7720,Woodstock Express,False,0,2025-05-13T18:40:22.000Z,California's Great America,United States,North America,37.397799,-121.974717
2,2025-05-13 18:41:52.601463+00:00,57,7660,The Pumpkin Patch,False,0,2025-05-13T18:40:22.000Z,California's Great America,United States,North America,37.397799,-121.974717
3,2025-05-13 18:41:52.601463+00:00,57,7668,Xtreme Skyflyer,False,0,2025-05-13T18:40:22.000Z,California's Great America,United States,North America,37.397799,-121.974717
4,2025-05-13 18:41:52.601463+00:00,57,13057,Tide Pool Slides,False,0,2025-05-13T18:40:22.000Z,California's Great America,United States,North America,37.397799,-121.974717
...,...,...,...,...,...,...,...,...,...,...,...,...
9515,2025-05-14 12:12:39.894646+00:00,274,7991,Westernland Shootin' Gallery,False,0,2025-05-14T12:07:54.000Z,Tokyo Disneyland,Japan,Asia,35.634848,139.879295
9516,2025-05-14 12:12:40.026123+00:00,275,8034,Venetian Gondolas,False,0,2025-05-14T12:08:09.000Z,Tokyo DisneySea,Japan,Asia,35.627055,139.889097
9517,2025-05-14 12:12:40.172345+00:00,28,10846,Spider-Man W.E.B. Adventure Single Rider,True,5,2025-05-14T12:10:04.000Z,Walt Disney Studios Paris,France,Europe,48.8673,2.779008
9518,2025-05-14 12:12:40.172345+00:00,28,40,The Twilight Zone Tower of Terror,True,30,2025-05-14T12:10:04.000Z,Walt Disney Studios Paris,France,Europe,48.8673,2.779008


In [255]:
wait_time_df = full_df[full_df['is_open']].filter(['ride_id', 'ride_name', 'park_name', 'country', 'wait_time'])
wait_time_df

,ride_id,ride_name,park_name,country,wait_time
5,6649,Yukon Striker,Canada's Wonderland,Canada,0
6,7645,White Water Canyon,Canada's Wonderland,Canada,0
7,7664,Woodstock Whirlybirds,Canada's Wonderland,Canada,0
8,5870,WindSeeker,Canada's Wonderland,Canada,0
9,7672,White Water Bay,Canada's Wonderland,Canada,0
...,...,...,...,...,...
9513,2998,Siren's Revenge,Shanghai Disney Resort,China,5
9514,12392,Zootopia: Hot Pursuit,Shanghai Disney Resort,China,50
9517,10846,Spider-Man W.E.B. Adventure Single Rider,Walt Disney Studios Paris,France,5
9518,40,The Twilight Zone Tower of Terror,Walt Disney Studios Paris,France,30


In [248]:
wait_time_df = full_df.filter(['ride_id', 'wait_time'])
wait_time_df

,ride_id,wait_time
0,5852,0
1,7720,0
2,7660,0
3,7668,0
4,13057,0
...,...,...
9515,7991,0
9516,8034,0
9517,10846,5
9518,40,30


In [253]:
wait_time_df.reset_index(drop=True, inplace=True)
wait_time_df = wait_time_df.groupby(['ride_id']).agg(
    avg_wait_time=('wait_time', 'mean'),
    min_wait_time=('wait_time', 'min'),
    max_wait_time=('wait_time', 'max')
)
wait_time_df


KeyError: 'ride_id'

In [231]:
parks_by_country_df = parks_df.filter(['park_id', 'country', 'continent'])
parks_by_country_df


,park_id,country,continent
0,57,United States,North America
1,58,Canada,North America
2,59,United States,North America
3,50,United States,North America
4,69,United States,North America
...,...,...,...
128,5,United States,North America
129,30,China,Asia
130,274,Japan,Asia
131,275,Japan,Asia


In [232]:
parks_by_country_df = parks_by_country_df.reset_index()
parks_by_country_df = parks_by_country_df.groupby('country').agg({
    'park_id': 'count',
    'continent': lambda x: sorted(set(x))[0]
}).rename(columns={'park_id': 'num_of_parks'}).reset_index()
    
parks_by_country_df.to_csv('parks_by_country.csv', index=False)


In [233]:
names_cols = parks_by_country_df.pivot(index='country', columns='name_number', values='park_name')
names_cols.columns = [f'park_name_{i}' for i in names_cols.columns]
names_cols

KeyError: 'name_number'

In [210]:
meta = parks_by_country_df.groupby('country').agg({
    'id': 'nunique',
    'continent': lambda x: sorted(set(x))[0]  # pick one (or check uniqueness)
}).rename(columns={'id': 'unique_id_count'}).reset_index()
meta


,country,unique_id_count,continent
0,Austria,1,Europe
1,Belgium,4,Europe
2,Brazil,1,South America
3,Canada,2,North America
4,China,8,Asia
5,Denmark,3,Europe
6,England,8,Europe
7,France,9,Europe
8,Germany,8,Europe
9,Hong Kong,1,Asia


In [212]:
parks_by_country_separated_df = pd.merge(meta, names_cols.reset_index(), on='country')
parks_by_country_separated_df.to_csv('parks_by_country_separated.csv', index=False)

In [251]:
avg_wait_time_by_ride_df = wait_time_df.drop(columns=['min_wait_time', 'max_wait_time'])
avg_wait_time_by_ride_df.reset_index(inplace=True)

In [94]:
ride_names_df = full_df.filter(['ride_id', 'ride_name', 'name'])
ride_names_df = ride_names_df.set_index('ride_id')
ride_names_df = ride_names_df.drop_duplicates()
ride_names_df 


,ride_name,name
ride_id,,
5852,RailBlazer,California's Great America
7720,Woodstock Express,California's Great America
7660,The Pumpkin Patch,California's Great America
7668,Xtreme Skyflyer,California's Great America
13057,Tide Pool Slides,California's Great America
...,...,...
7281,Toy Soldiers Parachute Drop Single Rider,Walt Disney Studios Paris
10990,​Zuma's Zoomers,Movie Park Germany
1428,Riptide Rescue,Seaworld San Diego


In [243]:
full_df['last_updated'] = pd.to_datetime(full_df['last_updated'], errors='coerce')
def format_date(dt):
    day = dt.day
    if 11 <= day <= 13:
        suffix = 'th'
    else:
        suffix = {1: 'st', 2: 'nd', 3: 'rd'}.get(day % 10, 'th')
    return f"{day}{suffix} {dt.strftime('%B %Y')}"

# Apply formatting
full_df['date'] = full_df['last_updated'].apply(format_date)
full_df

,timestamp,park_id,ride_id,ride_name,is_open,wait_time,last_updated,park_name,country,continent,latitude,longitude,date
0,2025-05-13 18:41:52.601463+00:00,57,5852,RailBlazer,False,0,2025-05-13 18:40:22+00:00,California's Great America,United States,North America,37.397799,-121.974717,13th May 2025
1,2025-05-13 18:41:52.601463+00:00,57,7720,Woodstock Express,False,0,2025-05-13 18:40:22+00:00,California's Great America,United States,North America,37.397799,-121.974717,13th May 2025
2,2025-05-13 18:41:52.601463+00:00,57,7660,The Pumpkin Patch,False,0,2025-05-13 18:40:22+00:00,California's Great America,United States,North America,37.397799,-121.974717,13th May 2025
3,2025-05-13 18:41:52.601463+00:00,57,7668,Xtreme Skyflyer,False,0,2025-05-13 18:40:22+00:00,California's Great America,United States,North America,37.397799,-121.974717,13th May 2025
4,2025-05-13 18:41:52.601463+00:00,57,13057,Tide Pool Slides,False,0,2025-05-13 18:40:22+00:00,California's Great America,United States,North America,37.397799,-121.974717,13th May 2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9515,2025-05-14 12:12:39.894646+00:00,274,7991,Westernland Shootin' Gallery,False,0,2025-05-14 12:07:54+00:00,Tokyo Disneyland,Japan,Asia,35.634848,139.879295,14th May 2025
9516,2025-05-14 12:12:40.026123+00:00,275,8034,Venetian Gondolas,False,0,2025-05-14 12:08:09+00:00,Tokyo DisneySea,Japan,Asia,35.627055,139.889097,14th May 2025
9517,2025-05-14 12:12:40.172345+00:00,28,10846,Spider-Man W.E.B. Adventure Single Rider,True,5,2025-05-14 12:10:04+00:00,Walt Disney Studios Paris,France,Europe,48.8673,2.779008,14th May 2025
9518,2025-05-14 12:12:40.172345+00:00,28,40,The Twilight Zone Tower of Terror,True,30,2025-05-14 12:10:04+00:00,Walt Disney Studios Paris,France,Europe,48.8673,2.779008,14th May 2025


In [ ]:

wait_time_df = full_df.filter(['continent', 'ride_id', 'ride_name', 'wait_time', 'name'])
wait_time_df 

,continent,ride_id,ride_name,wait_time,name
0,North America,5852,RailBlazer,0,California's Great America
1,North America,7720,Woodstock Express,0,California's Great America
2,North America,7660,The Pumpkin Patch,0,California's Great America
3,North America,7668,Xtreme Skyflyer,0,California's Great America
4,North America,13057,Tide Pool Slides,0,California's Great America
...,...,...,...,...,...
9515,Asia,7991,Westernland Shootin' Gallery,0,Tokyo Disneyland
9516,Asia,8034,Venetian Gondolas,0,Tokyo DisneySea
9517,Europe,10846,Spider-Man W.E.B. Adventure Single Rider,5,Walt Disney Studios Paris
9518,Europe,40,The Twilight Zone Tower of Terror,30,Walt Disney Studios Paris


In [245]:
# Find the row with the maximum wait time for each continent
max_wait_time_per_continent = full_df.loc[
    full_df.groupby("continent")["wait_time"].idxmax()
]
max_wait_time_per_continent = max_wait_time_per_continent.drop(columns=['ride_id', 'latitude', 'longitude', 'park_id', 'timestamp', 'is_open', 'last_updated'])
max_wait_time_per_continent = max_wait_time_per_continent.set_index('continent')
max_wait_time_per_continent


,ride_name,wait_time,park_name,country,date
continent,,,,,
Asia,Zootopia: Hot Pursuit,80,Shanghai Disney Resort,China,14th May 2025
Europe,Princess Pavilion,170,Disneyland Park Paris,France,14th May 2025
North America,Star Wars: Rise of the Resistance,85,Disneyland,United States,13th May 2025
South America,Xícaras Malucas,15,Beto Carrero World,Brazil,13th May 2025


In [246]:
# Display the result
max_wait_time_per_continent.to_csv('max_wait_time_per_continent.csv', index=True)

In [252]:
avg_wait_time_df = wait_time_df.groupby(['continent', 'ride_id', 'ride_name', 'name'], as_index=False).agg(
    avg_wait_time=('wait_time', 'mean')
)
avg_wait_time_df

KeyError: 'continent'

In [172]:

# Select only the required columns
result_df = avg_wait_time_df[['continent', 'name', 'ride_name', 'avg_wait_time']]

# Display the resulting DataFrame
result_df.to_csv("avg_wait_time_by_ride.csv", index=False)


In [ ]:
avg_wait_time_by_ride_df.to_csv("avg_wait_time_by_ride.csv", index=True)

In [95]:
ride_time_df = wait_time_df.merge(ride_names_df, left_on='ride_id', right_index=True, how='left')
ride_time_df

,avg_wait_time,min_wait_time,max_wait_time,ride_name,name
ride_id,,,,,
12,1.00,0,5,Main Street Vehicles,Disneyland Park Paris
24,44.00,0,170,Princess Pavilion,Disneyland Park Paris
28,0.00,0,0,Thunder Mesa Riverboat Landing,Disneyland Park Paris
40,18.75,0,60,The Twilight Zone Tower of Terror,Walt Disney Studios Paris
64,0.00,0,0,Tiger Rock,Chessington World of Adventures
...,...,...,...,...,...
14689,6.50,0,25,Yoshi's Adventure™,Epic Universe
14690,18.00,0,75,Stardust Racers,Epic Universe
14696,0.00,0,0,Harry Potter and the Battle at the Ministry™ S...,Epic Universe


In [48]:

average_wait_time_df = wait_time_df.groupby('ride_name', as_index=False)['wait_time'].mean()
average_wait_time_df 

,ride_name,wait_time
0,ALL NEW for 2025: Splash Island,0.000000
1,ALL-NEW 2024! Sky Striker,0.000000
2,Alpengeist,0.000000
3,Arena of Football - Be Part of It!,0.090909
4,Atlantica SuperSplash,0.090909
...,...,...
416,​Zuma's Zoomers,0.000000
417,缆车熊猫乐园站（环园线）,2.272727
418,风暴双响炮,2.272727
419,鹦鹉过山车,8.636364


In [49]:
map_df = full_df.filter(['name', 'latitude', 'longitude'])
map_df = map_df.drop_duplicates(subset=['name', 'latitude', 'longitude'])
map_df = map_df.rename(columns={'name': 'park_name'})
map_df = map_df.dropna(subset=['latitude', 'longitude'])
map_df

,park_name,latitude,longitude
0,Lotte World,37.51136,127.099768
5,PanoraMagique,48.86901442,2.78646006
10,Paultons Park,50.948542,-1.552557
14,Grona Lund,59.323333,18.096667
19,Parque Warner Madrid,40.23,-3.5937
...,...,...,...
450,Gardaland,45.4550142,10.7137527
451,Dorney Park,40.577942,-75.531528
456,Kings Dominion,37.84,-77.445
464,Chessington World of Adventures,51.3478981,-0.3173283


In [131]:
park_time_df = full_df.filter(['id', 'name', 'wait_time'])
average_park_time_df = park_time_df.groupby('name', as_index=False)['wait_time'].mean()
average_park_time_df = average_park_time_df.rename(columns={'name': 'park_name'})
def categorize_wait_time(w_time):
    if w_time < 5:
        return '100'
    elif 5 <= w_time < 15:
        return '10000'
    else:
        return '100000'
average_park_time_df["wait_time_category"] = average_park_time_df["wait_time"].apply(categorize_wait_time)
average_park_time_df 

,park_name,wait_time,wait_time_category
0,Adventureland Resort,0.000000,100
1,Alton Towers,4.500000,100
2,Animal Kingdom,3.285714,100
3,Avonturenpark Hellendoorn,0.000000,100
4,Bellewaerde,2.000000,100
...,...,...,...
116,Walibi Belgium,0.000000,100
117,Walibi Holland,0.888889,100
118,Walibi Rhône-Alpes,1.500000,100
119,Walt Disney Studios Paris,10.083333,10000


In [133]:
wait_map_df = average_park_time_df.merge(map_df, on='park_name', how='left')
wait_map_df = wait_map_df.dropna(subset=['latitude', 'longitude'])
wait_map_df
wait_map_df.to_csv("wait_map.csv", index=False)


In [266]:
open_rides = full_df[full_df['is_open']].filter(['ride_id','wait_time'])
open_rides

,ride_id,wait_time
5,6649,0
6,7645,0
7,7664,0
8,5870,0
9,7672,0
...,...,...
9513,2998,5
9514,12392,50
9517,10846,5
9518,40,30


In [268]:
    # Group by ride_id and calculate average wait time
avg_wait_by_ride = open_rides.groupby('ride_id').agg(
    avg_wait_time=('wait_time', 'mean')
)
    # Merge with the original dataframe
df = pd.merge(full_df, avg_wait_by_ride, on='ride_id', how='left')
df

,timestamp,park_id,ride_id,ride_name,is_open,wait_time,last_updated,park_name,country,continent,latitude,longitude,date,avg_wait_time
0,2025-05-13 18:41:52.601463+00:00,57,5852,RailBlazer,False,0,2025-05-13 18:40:22+00:00,California's Great America,United States,North America,37.397799,-121.974717,13th May 2025,NaN
1,2025-05-13 18:41:52.601463+00:00,57,7720,Woodstock Express,False,0,2025-05-13 18:40:22+00:00,California's Great America,United States,North America,37.397799,-121.974717,13th May 2025,NaN
2,2025-05-13 18:41:52.601463+00:00,57,7660,The Pumpkin Patch,False,0,2025-05-13 18:40:22+00:00,California's Great America,United States,North America,37.397799,-121.974717,13th May 2025,NaN
3,2025-05-13 18:41:52.601463+00:00,57,7668,Xtreme Skyflyer,False,0,2025-05-13 18:40:22+00:00,California's Great America,United States,North America,37.397799,-121.974717,13th May 2025,NaN
4,2025-05-13 18:41:52.601463+00:00,57,13057,Tide Pool Slides,False,0,2025-05-13 18:40:22+00:00,California's Great America,United States,North America,37.397799,-121.974717,13th May 2025,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9515,2025-05-14 12:12:39.894646+00:00,274,7991,Westernland Shootin' Gallery,False,0,2025-05-14 12:07:54+00:00,Tokyo Disneyland,Japan,Asia,35.634848,139.879295,14th May 2025,5.00
9516,2025-05-14 12:12:40.026123+00:00,275,8034,Venetian Gondolas,False,0,2025-05-14 12:08:09+00:00,Tokyo DisneySea,Japan,Asia,35.627055,139.889097,14th May 2025,22.00
9517,2025-05-14 12:12:40.172345+00:00,28,10846,Spider-Man W.E.B. Adventure Single Rider,True,5,2025-05-14 12:10:04+00:00,Walt Disney Studios Paris,France,Europe,48.8673,2.779008,14th May 2025,8.00
9518,2025-05-14 12:12:40.172345+00:00,28,40,The Twilight Zone Tower of Terror,True,30,2025-05-14 12:10:04+00:00,Walt Disney Studios Paris,France,Europe,48.8673,2.779008,14th May 2025,31.25


In [ ]:
df = pd.read_csv("../etl/data/outputs/average_wait.csv")
df